In [ ]:
!pip install tensorflow

In [1]:
#Importar librerias necesarias
import sys
import os
import itertools

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

K.clear_session()



In [2]:
#Importar el dataset
data_entrenamiento = 'C:/Users/jamer/Documents/Detector_plagas_cnn/dataset/train''
data_validacion = 'C:/Users/jamer/Documents/Detector_plagas_cnn/dataset/test'

#numero de evaluaciones del modelo
epocas=20
#tamanio imagenes
longitud, altura = 150, 100
batch_size = 15
#Parametros
pasos = 100
validation_steps = 10
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3
lr = 0.0004


##Preparamos nuestras imagenes

#Generar el entrenamiento modificando parametros de la imagen
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=10,  # Agregar rotación
    width_shift_range=0.2,  # Cambio horizontal
    height_shift_range=0.2,  # Cambio vertical
    brightness_range=[0.8, 1.2],  # Cambio de brillo
    fill_mode='nearest'  # Modo de relleno
)
#Reescalar la imagen de 0 a 1 para reducir el error
test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

entrenamiento_generador = itertools.cycle(entrenamiento_generador)

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

#Definir redes

cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])



#Generar el modelo CNN
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)




Found 747 images belonging to 3 classes.
Found 122 images belonging to 3 classes.


<ipython-input-2-cf14e97136e2>:70: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(


Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: 1.0208 - accuracy: 0.5924

100/100 [==============================] - 168s 2s/step - loss: 1.0208 - accuracy: 0.5924 - val_loss: 0.8369 - val_accuracy: 0.6230
Epoch 2/20
100/100 [==============================] - 45s 448ms/step - loss: 0.8056 - accuracy: 0.6693
Epoch 3/20
100/100 [==============================] - 46s 462ms/step - loss: 0.8008 - accuracy: 0.6754
Epoch 4/20
100/100 [==============================] - 45s 446ms/step - loss: 0.7319 - accuracy: 0.7015
Epoch 5/20
100/100 [==============================] - 45s 447ms/step - loss: 0.6871 - accuracy: 0.7129
Epoch 6/20
100/100 [==============================] - 45s 451ms/step - loss: 0.6614 - accuracy: 0.7363
Epoch 7/20
100/100 [==============================] - 46s 456ms/step - loss: 0.6476 - accuracy: 0.7262
Epoch 8/20
100/100 [==============================] - 45s 455ms/step - loss: 0.6382 - accuracy: 0.7303
Epoch 9/20
100/100 [==============================] - 45s 451ms/step - loss: 0.6143 - accuracy: 0.7477
Epoch 10/20
100/100 [=======================

In [9]:
#Guarda el modelo y los pesos en la ruta deseada
target_dir = '/content/drive/MyDrive/archive'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('C:/Users/jamer/Documents/Detector_plagas_cnn/modelo/modelo.h5')
cnn.save_weights('C:/Users/jamer/Documents/Detector_plagas_cnn/modelo/pesos.h5')